In [1]:
# Importing necessary libraries.

from bs4 import BeautifulSoup as soup
from requests import get
import pandas as pd
import requests
import re

In [2]:
# Creating empty lists in-order to append data scrapped from url's.
# The number of lists depends on the number of features you are extracting from the url.

bhk=[]
location=[]
price=[]
price_per_sqft=[]
area_in_sqft=[]
construction=[]
bathroom=[]
type_of_sale=[]
furnishing=[]

In [3]:
%%time 
#Returns time take for execution of the cell.


n_pages = 0
for page in range(1,6558):  #The no. of pages you wanted to scrap considering pages available in the url.
    n_pages += 1
    url = 'https://www.makaan.com/listings?propertyType=apartment&listingType=buy&pageType=CITY_URLS&cityName=Mumbai&cityId=18&templateId=MAKAAN_CITY_LISTING_BUY '+'&page='+str(page)
    # This for gets us into next page after every iteration.
    r = requests.get(url) # Here r is response data.
    page_html = soup(r.text, 'html.parser') 
    house_containers = page_html.find_all("li",class_="cardholder") # Class containing overall data of a property.
    for data in house_containers:
################################   TITLE   ########################################  
  
        bhk_=data.find_all("div",class_="title-line")	 # Class containing title name of a property.
        for i in bhk_:
            d=i.get_text(strip=True)[0]
            bhk.append(d) # Appending the extracted data into a list.
            
################################ LOCATION  #######################################

        location_=data.find_all("a",class_="loclink")
        for i in location_:
            d=i.text
            r= d.split(',',)[0] # Splitting the obtained text and returning the first element of text.
            location.append(r)
            
################################    PRICE    ######################################

        cost=data.find_all("td",class_="price")
        for i in cost:
            d=i.text
            if 'L' in d:
                a = d.split()[0]
                price.append(float(a)) # Type conversion of obtained data.
            elif 'Cr' in d:
                b = d.split()[0]
                price.append(float(b) * 100) # Coverting CRORES into LAKHS.
            else:  # If no value found in the class return value 0.
                e=0
                price.append(float(e)) 
                
############################   PRICE PER SQFT   ###################################

        rate_sqft=data.find_all('td',class_="lbl rate")
        for i in rate_sqft:
            d=i.text
            res= d.split('/',)[0]
            r=re.sub(",","" ,res) # Substitute no character in the place of ,(comma).
            price_per_sqft.append(r)
            
############################   AREA IN SQFT    ####################################

        area_sqft=data.find_all('td',class_="size")
        for i in area_sqft:
            d=i.text
            area_in_sqft.append(d)
            
########################   CONSTRUCTION STATUS    #################################

        construction_=data.find_all("td",class_="val")
        for i in construction_:
            d=i.text
            construction.append(d)

            

##########################   TYPE_OF_SALE    #####################################


        type_of_sale_=data.find_all("ul",class_="listing-details")
        for i in type_of_sale_:
            d=i.text
            type_of_sale.append(d)

            
   

            
print('You scraped {} pages containing {} properties.'.format(n_pages, len(bhk)))
# Returns the number of pages scrapped and time taken for scarpping those pages.

You scraped 6557 pages containing 131120 properties.
Wall time: 3h 26min 49s
Compiler : 854 ms


In [4]:
#Creating dataframes for each column
df1 = pd.DataFrame(bhk,columns=['BHK'])
df2 = pd.DataFrame(location,columns=['location'])
df3 = pd.DataFrame(price,columns=['price(L)'])
df4 = pd.DataFrame(price_per_sqft,columns=['rate_persqft'])
df5 = pd.DataFrame(area_in_sqft,columns=['area_insqft'])
df6 = pd.DataFrame(construction,columns=['construction_status'])
df7 = pd.DataFrame(type_of_sale,columns=['type_of_sale'])

In [5]:
#Concatinating the dataframes
df = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=1) 

In [6]:
df

,BHK,location,price(L),rate_persqft,area_insqft,construction_status,type_of_sale
0,1,Panvel,18.49,7034,263,Under Construction,Possession by Mar 2024New
1,1,Dombivali,40.00,11869,337,Under Construction,Possession by Nov 2023New
2,2,Koper Khairane,149.00,22513,664,Under Construction,Possession by Feb 2023New
3,2,Dombivali,49.00,7802,628,Under Construction,Possession by Sep 2021New
4,1,Jogeshwari West,96.00,14222,675,Under Construction,Possession by Nov 20232 BathroomsNew
...,...,...,...,...,...,...,...
131115,1,Ghatkopar East,45.00,15000,300,Ready to move,121 - 122 years old1 BathroomsResale
131116,1,Ghatkopar,80.00,14545,550,Ready to move,121 - 122 years old1 BathroomsResale
131117,2,Ghatkopar,170.00,25185,675,Ready to move,121 - 122 years old2 BathroomsResale
131118,2,Ghatkopar,17.00,1789,950,Ready to move,121 - 122 years old2 BathroomsResale


In [9]:
word_list=['New','Resale']
df['Sale_type'] = df.type_of_sale.str.extract('({})'.format('|'.join(word_list)), 
                        flags=re.IGNORECASE, expand=False).str.lower().fillna('')
df['Sale_type'].head(10)

0    new
1    new
2    new
3    new
4    new
5    new
6    new
7    new
8    new
9    new
Name: Sale_type, dtype: object

In [11]:
word_list=['1 Bathrooms','2 Bathrooms','3 Bathrooms','4 Bathrooms','5 Bathrooms','6 Bathrooms','7 Bathrooms','8 Bathrooms','9 Bathrooms']
df['Bathroom'] = df.type_of_sale.str.extract('({})'.format('|'.join(word_list)), 
                        flags=re.IGNORECASE, expand=False).str.lower().fillna('')
df['Bathroom'].head(10)

0               
1               
2               
3               
4    2 bathrooms
5               
6               
7    2 bathrooms
8               
9               
Name: Bathroom, dtype: object

In [12]:
#Exporting to csv
df.to_csv(r'D:\Datascienceprojects\Rental Prediction\house_scrape.csv' ,index=False,header=True)